In [0]:
%scala
// in Scala
val person = Seq(
    (0, "Bill Chambers", 0, Seq(100)),
    (1, "Matei Zaharia", 1, Seq(500, 250, 100)),
    (2, "Michael Armbrust", 1, Seq(250, 100)))
  .toDF("id", "name", "graduate_program", "spark_status")
val graduateProgram = Seq(
    (0, "Masters", "School of Information", "UC Berkeley"),
    (2, "Masters", "EECS", "UC Berkeley"),
    (1, "Ph.D.", "EECS", "UC Berkeley"))
  .toDF("id", "degree", "department", "school")
val sparkStatus = Seq(
    (500, "Vice President"),
    (250, "PMC Member"),
    (100, "Contributor"))
  .toDF("id", "status")


person: org.apache.spark.sql.DataFrame = [id: int, name: string ... 2 more fields]
graduateProgram: org.apache.spark.sql.DataFrame = [id: int, degree: string ... 2 more fields]
sparkStatus: org.apache.spark.sql.DataFrame = [id: int, status: string]

In [0]:
%scala
/*person = spark.createDataFrame(
  [
    (0, "Bill Chambers", 0, [100]),
    (1, "Matei Zaharia", 1, [500, 250, 100]),
    (2, "Michael Armbrust", 1, [250, 100])
  ], ["id", "name", "graduate_program", "spark_status"]
)
I gave up on the python part ;)
*/

In [0]:
%scala
person.createOrReplaceTempView("person")
graduateProgram.createOrReplaceTempView("graduateProgram")
sparkStatus.createOrReplaceTempView("sparkStatus")


In [0]:
%scala
val person = spark.sql("SELECT * FROM person")
display(person)

id,name,graduate_program,spark_status
0,Bill Chambers,0,List(100)
1,Matei Zaharia,1,"List(500, 250, 100)"
2,Michael Armbrust,1,"List(250, 100)"


In [0]:
%scala
display(spark.sql("SELECT * FROM graduateProgram"))

id,degree,department,school
0,Masters,School of Information,UC Berkeley
2,Masters,EECS,UC Berkeley
1,Ph.D.,EECS,UC Berkeley


In [0]:
%scala
display(spark.sql("SELECT * FROM sparkStatus"))

id,status
500,Vice President
250,PMC Member
100,Contributor


In [0]:
%scala
// in Scala
val joinExpression = person.col("graduate_program") === graduateProgram.col("id")


joinExpression: org.apache.spark.sql.Column = `=`(graduate_program, id)

In [0]:
%scala
// in Scala
val wrongJoinExpression = person.col("name") === graduateProgram.col("school")


wrongJoinExpression: org.apache.spark.sql.Column = `=`(name, school)

In [0]:
%scala
person.join(graduateProgram, joinExpression).show()


+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
 id| name|graduate_program| spark_status| id| degree| department| school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
 0| Bill Chambers| 0| [100]| 0|Masters|School of Informa...|UC Berkeley|
 2|Michael Armbrust| 1| [250, 100]| 1| Ph.D.| EECS|UC Berkeley|
 1| Matei Zaharia| 1|[500, 250, 100]| 1| Ph.D.| EECS|UC Berkeley|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+

In [0]:
%scala
person.join(graduateProgram, wrongJoinExpression).show()

+---+----+----------------+------------+---+------+----------+------+
 id|name|graduate_program|spark_status| id|degree|department|school|
+---+----+----------------+------------+---+------+----------+------+
+---+----+----------------+------------+---+------+----------+------+

In [0]:
%scala
// in Scala
var joinType = "inner"
person.join(graduateProgram, joinExpression, joinType).explain

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- == Initial Plan ==
 BroadcastHashJoin [graduate_program#25], [id#44], Inner, BuildLeft, false, true
 :- Exchange SinglePartition, EXECUTOR_BROADCAST, [plan_id=598]
 : +- LocalTableScan [id#23, name#24, graduate_program#25, spark_status#26]
 +- LocalTableScan [id#44, degree#45, department#46, school#47]


joinType: String = inner

In [0]:
%scala
joinType = "outer"
person.join(graduateProgram, joinExpression, joinType).show()


+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
 id| name|graduate_program| spark_status| id| degree| department| school|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
 0| Bill Chambers| 0| [100]| 0|Masters|School of Informa...|UC Berkeley|
 1| Matei Zaharia| 1|[500, 250, 100]| 1| Ph.D.| EECS|UC Berkeley|
 2|Michael Armbrust| 1| [250, 100]| 1| Ph.D.| EECS|UC Berkeley|
NULL| NULL| NULL| NULL| 2|Masters| EECS|UC Berkeley|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+

joinType: String = outer

In [0]:
%scala
joinType = "left_outer"
graduateProgram.join(person, joinExpression, joinType).show()


+---+-------+--------------------+-----------+----+----------------+----------------+---------------+
 id| degree| department| school| id| name|graduate_program| spark_status|
+---+-------+--------------------+-----------+----+----------------+----------------+---------------+
 0|Masters|School of Informa...|UC Berkeley| 0| Bill Chambers| 0| [100]|
 2|Masters| EECS|UC Berkeley|NULL| NULL| NULL| NULL|
 1| Ph.D.| EECS|UC Berkeley| 2|Michael Armbrust| 1| [250, 100]|
 1| Ph.D.| EECS|UC Berkeley| 1| Matei Zaharia| 1|[500, 250, 100]|
+---+-------+--------------------+-----------+----+----------------+----------------+---------------+

joinType: String = left_outer

In [0]:
%scala
joinType = "right"
person.join(graduateProgram, joinExpression, joinType).show()


+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
 id| name|graduate_program| spark_status| id| degree| department| school|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
 0| Bill Chambers| 0| [100]| 0|Masters|School of Informa...|UC Berkeley|
NULL| NULL| NULL| NULL| 2|Masters| EECS|UC Berkeley|
 2|Michael Armbrust| 1| [250, 100]| 1| Ph.D.| EECS|UC Berkeley|
 1| Matei Zaharia| 1|[500, 250, 100]| 1| Ph.D.| EECS|UC Berkeley|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+

joinType: String = right

In [0]:
%scala
joinType = "left_semi"
graduateProgram.join(person, joinExpression, joinType).show()


+---+-------+--------------------+-----------+
 id| degree| department| school|
+---+-------+--------------------+-----------+
 0|Masters|School of Informa...|UC Berkeley|
 1| Ph.D.| EECS|UC Berkeley|
+---+-------+--------------------+-----------+

joinType: String = left_semi

In [0]:
%scala
// in Scala
val gradProgram2 = graduateProgram.union(Seq(
    (0, "Masters", "Duplicated Row", "Duplicated School")).toDF())

gradProgram2.createOrReplaceTempView("gradProgram2")


gradProgram2: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [id: int, degree: string ... 2 more fields]

In [0]:
%scala
gradProgram2.join(person, joinExpression, joinType).show()


+---+-------+--------------------+-----------------+
 id| degree| department| school|
+---+-------+--------------------+-----------------+
 0|Masters|School of Informa...| UC Berkeley|
 1| Ph.D.| EECS| UC Berkeley|
 0|Masters| Duplicated Row|Duplicated School|
+---+-------+--------------------+-----------------+

In [0]:
%scala
joinType = "left_anti"
graduateProgram.join(person, joinExpression, joinType).show()


+---+-------+----------+-----------+
 id| degree|department| school|
+---+-------+----------+-----------+
 2|Masters| EECS|UC Berkeley|
+---+-------+----------+-----------+

joinType: String = left_anti

In [0]:
%scala
joinType = "cross"
graduateProgram.join(person, joinExpression, joinType).show()


+---+-------+--------------------+-----------+---+----------------+----------------+---------------+
 id| degree| department| school| id| name|graduate_program| spark_status|
+---+-------+--------------------+-----------+---+----------------+----------------+---------------+
 0|Masters|School of Informa...|UC Berkeley| 0| Bill Chambers| 0| [100]|
 1| Ph.D.| EECS|UC Berkeley| 2|Michael Armbrust| 1| [250, 100]|
 1| Ph.D.| EECS|UC Berkeley| 1| Matei Zaharia| 1|[500, 250, 100]|
+---+-------+--------------------+-----------+---+----------------+----------------+---------------+

joinType: String = cross

In [0]:
%scala

graduateProgram.crossJoin(person).show()

+---+-------+--------------------+-----------+---+----------------+----------------+---------------+
 id| degree| department| school| id| name|graduate_program| spark_status|
+---+-------+--------------------+-----------+---+----------------+----------------+---------------+
 0|Masters|School of Informa...|UC Berkeley| 0| Bill Chambers| 0| [100]|
 0|Masters|School of Informa...|UC Berkeley| 1| Matei Zaharia| 1|[500, 250, 100]|
 0|Masters|School of Informa...|UC Berkeley| 2|Michael Armbrust| 1| [250, 100]|
 2|Masters| EECS|UC Berkeley| 0| Bill Chambers| 0| [100]|
 2|Masters| EECS|UC Berkeley| 1| Matei Zaharia| 1|[500, 250, 100]|
 2|Masters| EECS|UC Berkeley| 2|Michael Armbrust| 1| [250, 100]|
 1| Ph.D.| EECS|UC Berkeley| 0| Bill Chambers| 0| [100]|
 1| Ph.D.| EECS|UC Berkeley| 1| Matei Zaharia| 1|[500, 250, 100]|
 1| Ph.D.| EECS|UC Berkeley| 2|Michael Armbrust| 1| [250, 100]|
+---+-------+--------------------+-----------+---+----------------+----------------+---------------+

In [0]:
%scala
graduateProgram.join(person, joinExpression, joinType).explain

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- == Initial Plan ==
 BroadcastHashJoin [id#44], [graduate_program#25], Cross, BuildRight, false, true
 :- LocalTableScan [id#44, degree#45, department#46, school#47]
 +- Exchange SinglePartition, EXECUTOR_BROADCAST, [plan_id=1052]
 +- LocalTableScan [id#23, name#24, graduate_program#25, spark_status#26]

In [0]:
%scala
graduateProgram.crossJoin(person).explain

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- == Initial Plan ==
 BroadcastNestedLoopJoin BuildRight, Cross, true
 :- LocalTableScan [id#44, degree#45, department#46, school#47]
 +- Exchange SinglePartition, EXECUTOR_BROADCAST, [plan_id=1072]
 +- LocalTableScan [id#23, name#24, graduate_program#25, spark_status#26]

In [0]:
%scala
import org.apache.spark.sql.functions.expr

person.withColumnRenamed("id", "personId")
  .join(sparkStatus, expr("array_contains(spark_status, id)")).show()


+--------+----------------+----------------+---------------+---+--------------+
personId| name|graduate_program| spark_status| id| status|
+--------+----------------+----------------+---------------+---+--------------+
 0| Bill Chambers| 0| [100]|100| Contributor|
 1| Matei Zaharia| 1|[500, 250, 100]|500|Vice President|
 1| Matei Zaharia| 1|[500, 250, 100]|250| PMC Member|
 1| Matei Zaharia| 1|[500, 250, 100]|100| Contributor|
 2|Michael Armbrust| 1| [250, 100]|250| PMC Member|
 2|Michael Armbrust| 1| [250, 100]|100| Contributor|
+--------+----------------+----------------+---------------+---+--------------+

import org.apache.spark.sql.functions.expr

### Duplikaty

In [0]:
%scala
val gradProgramDupe = graduateProgram.withColumnRenamed("id", "graduate_program")


gradProgramDupe: org.apache.spark.sql.DataFrame = [graduate_program: int, degree: string ... 2 more fields]

In [0]:
%scala
display(gradProgramDupe)

graduate_program,degree,department,school
0,Masters,School of Information,UC Berkeley
2,Masters,EECS,UC Berkeley
1,Ph.D.,EECS,UC Berkeley


In [0]:
%scala
val joinExpr = gradProgramDupe.col("graduate_program") === person.col("graduate_program")


joinExpr: org.apache.spark.sql.Column = `=`(graduate_program, graduate_program)

In [0]:
%scala
person.join(gradProgramDupe, joinExpr).show()


+---+----------------+----------------+---------------+----------------+-------+--------------------+-----------+
 id| name|graduate_program| spark_status|graduate_program| degree| department| school|
+---+----------------+----------------+---------------+----------------+-------+--------------------+-----------+
 0| Bill Chambers| 0| [100]| 0|Masters|School of Informa...|UC Berkeley|
 2|Michael Armbrust| 1| [250, 100]| 1| Ph.D.| EECS|UC Berkeley|
 1| Matei Zaharia| 1|[500, 250, 100]| 1| Ph.D.| EECS|UC Berkeley|
+---+----------------+----------------+---------------+----------------+-------+--------------------+-----------+

In [0]:
%scala

person.join(gradProgramDupe, joinExpr).select("graduate_program").show()
// oj


at org.apache.spark.sql.errors.QueryCompilationErrors$.ambiguousReferenceError(QueryCompilationErrors.scala:2627)
	at org.apache.spark.sql.catalyst.expressions.package$AttributeSeq.resolve(package.scala:407)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveChildren(LogicalPlan.scala:183)
	at org.apache.spark.sql.catalyst.analysis.ColumnResolutionHelper.$anonfun$resolveExpressionByPlanChildren$1(ColumnResolutionHelper.scala:534)
	at org.apache.spark.sql.catalyst.analysis.ColumnResolutionHelper.$anonfun$resolveExpression$3(ColumnResolutionHelper.scala:175)
	at org.apache.spark.sql.catalyst.analysis.package$.withPosition(package.scala:103)
	at org.apache.spark.sql.catalyst.analysis.ColumnResolutionHelper.$anonfun$resolveExpression$1(ColumnResolutionHelper.scala:182)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(origin.scala:85)
	at org.apache.spark.sql.catalyst.analysis.ColumnResolutionHelper.innerResolve$1(ColumnResolutionHelper.scala:145)
	at org.apache.spark.sql.catalyst.analysis.ColumnResolutionHelper.resolveExpression(ColumnResolutionHelper.scala:212)
	at org.apache.spark.sql.catalyst.analysis.ColumnResolutionHelper.resolveExpressionByPlanChildren(ColumnResolutionHelper.scala:541)
	at org.apache.spark.sql.catalyst.analysis.ColumnResolutionHelper.resolveExpressionByPlanChildren$(ColumnResolutionHelper.scala:527)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveReferences.resolveExpressionByPlanChildren(Analyzer.scala:2378)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveReferences$$anonfun$apply$17.$anonfun$applyOrElse$121(Analyzer.scala:2529)
	at scala.collection.TraversableLike.$anonfun$map$1(TraversableLike.scala:286)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at scala.collection.TraversableLike.map(TraversableLike.scala:286)
	at scala.collection.TraversableLike.map$(TraversableLike.scala:279)
	at scala.collection.AbstractTraversable.map(Traversable.scala:108)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveReferences$$anonfun$apply$17.applyOrElse(Analyzer.scala:2529)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveReferences$$anonfun$apply$17.applyOrElse(Analyzer.scala:2413)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.$anonfun$resolveOperatorsUpWithPruning$3(AnalysisHelper.scala:141)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(origin.scala:85)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.$anonfun$resolveOperatorsUpWithPruning$1(AnalysisHelper.scala:141)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.allowInvokingTransformsInAnalyzer(AnalysisHelper.scala:436)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperatorsUpWithPruning(AnalysisHelper.scala:137)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperatorsUpWithPruning$(AnalysisHelper.scala:133)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveOperatorsUpWithPruning(LogicalPlan.scala:40)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperatorsUp(AnalysisHelper.scala:114)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperatorsUp$(AnalysisHelper.scala:113)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveOperatorsUp(LogicalPlan.scala:40)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveReferences.apply(Analyzer.scala:2413)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveReferences.apply(Analyzer.scala:2378)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.$anonfun$execute$4(RuleExecutor.scala:327)
	at com.databricks.spark.util.FrameProfiler$.record(FrameProfiler.scala:94)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.$anonfun$execute$3(RuleExecutor.scala:327)
	at scala.collection

###Opcja 1

In [0]:
%scala
person.join(gradProgramDupe,"graduate_program").select("graduate_program").show()


+----------------+
graduate_program|
+----------------+
 0|
 1|
 1|
+----------------+

### Opcja 2 Drop after join

In [0]:
%scala
person.join(gradProgramDupe, joinExpr).drop(person.col("graduate_program")).select("graduate_program").show()

//person.join(gradProgramDupe, joinExpr).drop(person.col("graduate_program")).show()
  


+----------------+
graduate_program|
+----------------+
 0|
 1|
 1|
+----------------+

In [0]:
%scala
val joinExpr = person.col("graduate_program") === graduateProgram.col("id")
person.join(graduateProgram, joinExpr).drop(graduateProgram.col("id")).show()


+---+----------------+----------------+---------------+-------+--------------------+-----------+
 id| name|graduate_program| spark_status| degree| department| school|
+---+----------------+----------------+---------------+-------+--------------------+-----------+
 0| Bill Chambers| 0| [100]|Masters|School of Informa...|UC Berkeley|
 2|Michael Armbrust| 1| [250, 100]| Ph.D.| EECS|UC Berkeley|
 1| Matei Zaharia| 1|[500, 250, 100]| Ph.D.| EECS|UC Berkeley|
+---+----------------+----------------+---------------+-------+--------------------+-----------+

joinExpr: org.apache.spark.sql.Column = `=`(graduate_program, id)

### Opcja 3

In [0]:
%scala
val gradProgram3 = graduateProgram.withColumnRenamed("id", "grad_id")
val joinExpr = person.col("graduate_program") === gradProgram3.col("grad_id")
person.join(gradProgram3, joinExpr).show()


+---+----------------+----------------+---------------+-------+-------+--------------------+-----------+
 id| name|graduate_program| spark_status|grad_id| degree| department| school|
+---+----------------+----------------+---------------+-------+-------+--------------------+-----------+
 0| Bill Chambers| 0| [100]| 0|Masters|School of Informa...|UC Berkeley|
 2|Michael Armbrust| 1| [250, 100]| 1| Ph.D.| EECS|UC Berkeley|
 1| Matei Zaharia| 1|[500, 250, 100]| 1| Ph.D.| EECS|UC Berkeley|
+---+----------------+----------------+---------------+-------+-------+--------------------+-----------+

gradProgram3: org.apache.spark.sql.DataFrame = [grad_id: int, degree: string ... 2 more fields]
joinExpr: org.apache.spark.sql.Column = `=`(graduate_program, grad_id)